In [1]:
#General packages
import numpy as np
import emcee
import matplotlib.pyplot as plt

#Own packages
import My_Jampy
import model_data
from model_data import Global_Parameters as GP 
from model_data import Jampy_data as JP_data
from model_data import Autolens_data as AL_data
import emcee_probabilities
from emcee_probabilities import Probability 

#MPI packages and control paralelization
from time import perf_counter as clock
from multiprocessing import Pool
from schwimmbad import MPIPool
import time
import os

os.environ["OMP_NUM_THREADS"] = "1"

#Autolens Model packages

import autolens as al
import autolens.plot as aplt
#print("Pyautolens version:", al.__version__)

from pyprojroot import here

from time import perf_counter as clock

from astropy.cosmology import Planck15 as cosmo
from astropy.constants import G, M_sun, c
import astropy.units as u
workspace_path = str(here())
#print("Workspace Path: ", workspace_path)

In [2]:
# JAMPY MODEL

#Initializing the dynamic model
Jampy_Model = My_Jampy.Jam_axi_rms(ybin=JP_data.ybin, xbin=JP_data.xbin, beta=JP_data.beta, mbh=GP.mbh.value, distance=GP.distance.value, surf_lum=GP.surf_star_dat.value, sigma_lum=GP.sigma_star_dat_ARC.value, qobs_lum=GP.qstar_dat, surf_DM=GP.surf_DM_dat.value, sigma_DM=GP.sigma_DM_dat_ARC.value, qobs_DM=GP.qDM_dat, ml=GP.ML.value, goodBins=JP_data.goodBins, sigmapsf=JP_data.sigmapsf, rms=JP_data.rms, erms=JP_data.erms, pixsize=JP_data.pixsize, inc=GP.inc, quiet=False)

Jampy Class successfully initialized!!


In [3]:
# Pyautolens Model

### __Reading simulated data__

#Paths
dataset_type = "JAM+Pyautolens"
dataset_name = "Data"
dataset_path = f"/home/carlos/Documents/GitHub/Master-Degree/ESO325/Arcs Modelling/autolens_workspace/{dataset_type}/{dataset_name}"

#Load data
imaging = al.Imaging.from_fits(
    image_path=f"{dataset_path}/arcs_resized.fits",
    noise_map_path=f"{dataset_path}/noise_map_resized.fits",
    psf_path=f"{dataset_path}/psf.fits",
    pixel_scales=0.04,
)

#Load mask
mask_custom = al.Mask.from_fits(
    file_path=f"{dataset_path}/mask gui.fits", hdu=0, pixel_scales=imaging.pixel_scales
)

masked_imaging = al.MaskedImaging(imaging=imaging, mask=mask_custom)

#Plot
'''
aplt.Imaging.subplot_imaging(
    imaging=imaging, mask=mask_custom, include=aplt.Include(border=True)
)
'''

### __Defining the MGE mass model__

#Initializing the MGE  model for lens

mass_profile = al.mp.MGE(centre=(0.0, 0.0))                               #Defining the mass model
mass_profile.MGE_comps(M=AL_data.Total_Mass.value, sigma=AL_data.Total_sigma_RAD.value, q=AL_data.Total_q_proj.value, z_l=GP.z_lens, z_s=GP.z_source)        #Input data

mass_profile.MGE_Grid_parameters(masked_imaging.grid)               #Creating the parameter grid for the parallel calculation

#Lens Model
lens_galaxy = al.Galaxy(                                            
        redshift=GP.z_lens,
        mass=mass_profile,
        shear=al.mp.ExternalShear(elliptical_comps=(0,0)),
    )

In [4]:
### Priors
# parameter boundaries. [lower, upper]
boundary = {'inc': [70, 120], 'beta': [-5, 5], 'ml': [0.5, 15], 'log_mbh':[7, 11], 'qDM': [0.15, 1],
                'log_rho_s':[6, 13], 'mag_shear': [-0.2, 0.2], 'phi_shear': [-0.2, 0.2],
                'gamma': [-2, 2]
               }

# parameter gaussian priors. [mean, sigma]
prior = {'inc': [90, 10], 'beta': [0.0, 2], 'ml': [1.0, 5],'log_mbh':[8, 3], 'qDM': [0.5, 3e-1],
            'log_rho_s':[10, 4], 'mag_shear': [0, 0.1], 'phi_shear': [0, 0.1], 'gamma': [1.0, 0.5]
            }

In [10]:
ml = np.zeros((50,6))
ml[:] = np.array([9.5,8.5,3.8,3.4,3.2,2.8])
ml_noise = np.random.rand(50,6)

beta = np.zeros((50,7))
beta[:] = np.array([-0.6, -1.0, 0.34, -3.4, 0.39, -0.31, 0.36])
beta_noise = np.random.rand(50,7)

ml = ml + ml_noise                                                               #Between [2.8, 10.5]
beta = beta + beta_noise                                                         #Between [-3.4, 1.39]
inc = prior ['inc'][0] + (np.random.rand(50,1) -0.5)*10                          #Between [85, 95]
qDM = np.random.rand(50,1)*0.74+0.26                                             #Between [0.26, 1]
log_rho_s = np.random.rand(50,1)*prior['log_rho_s'][0]                           #Between [0, 10]
log_mbh =  np.random.rand(50,1)+9.0                                              #Between [9.0, 10]
mag_shear = (np.random.rand(50,1) - 0.5)*0.2                                     #Between [-0.1, 0.1]
phi_shear = (np.random.rand(50,1) - 0.5)*0.2                                     #Between [-0.1, 0.1]
gamma = (np.random.rand(50,1) - 0.5)*2                                           #Between [-1, 1]
    #50 walkers in a 21-D space
pos = np.append(ml, beta, axis=1)
pos = np.append(pos, inc, axis=1)
pos = np.append(pos, qDM, axis=1)
pos = np.append(pos, log_rho_s, axis=1)
pos = np.append(pos, log_mbh, axis=1)
pos = np.append(pos, mag_shear, axis=1)
pos = np.append(pos, phi_shear, axis=1)
pos = np.append(pos, gamma, axis=1)

nwalkers, ndim = pos.shape

In [6]:
pos = np.array([ 9.90519845e+00,  9.38761809e+00,  4.56670065e+00,  3.95540285e+00,
        4.09138376e+00,  3.03500598e+00, -1.23701618e-01, -5.95326160e-01,
        6.70712457e-01, -2.71696684e+00,  1.28491888e+00, -9.11095039e-02,
        6.93551706e-01,  8.50485764e+01,  5.45598952e-01,  8.21609655e+00,
        9.16380573e+00,  2.62978391e-02,  3.44317093e-02, -8.18626727e-01])

In [11]:
model = Probability(Jampy_Model=Jampy_Model, mass_profile=mass_profile, masked_imaging=masked_imaging)

In [16]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, model.log_probability)

In [17]:
sampler.run_mcmc(pos, 1, progress=True);

/home/carlos/anaconda3/lib/python3.7/site-packages/autogalaxy/profiles/mass_profiles/total_mass_profiles.py:98: RuntimeWarning: invalid value encountered in sqrt
  eta = np.sqrt(1 - q**2)
/home/carlos/anaconda3/lib/python3.7/site-packages/autogalaxy/profiles/mass_profiles/total_mass_profiles.py:120: RuntimeWarning: invalid value encountered in sqrt
  eta = np.sqrt(1 - q**2)


emcee: Exception while calling your likelihood function:
  params: [ 9.87569315e+00  8.89821362e+00  4.58614834e+00  3.59263004e+00
  4.04383537e+00  2.93268322e+00  3.87721023e-02 -9.70043145e-01
  5.82364422e-01 -3.10794883e+00  6.59058992e-01  3.65881878e-01
  5.76436395e-01  9.34566836e+01  5.73793773e-01  7.25652538e+00
  9.51231057e+00 -5.54143885e-02 -5.34149651e-02  7.51963055e-02]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/carlos/anaconda3/lib/python3.7/site-packages/emcee/ensemble.py", line 545, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/carlos/Documents/GitHub/Master-Degree/ESO325/Arcs Modelling/autolens_workspace/JAM+Pyautolens/Cluster_Codes/emcee_probabilities.py", line 266, in log_probability
    return lp + self.Pyautolens_log_likelihood(parsDic) + self.JAM_log_likelihood(parsDic)
  File "/home/carlos/Documents/GitHub/Master-Degree/ESO325/Arcs Modelling/autolens_workspace/JAM+Pyautolens/Cluster_Codes/emcee_probabilities.py", line 232, in Pyautolens_log_likelihood
    source_plane_grid = tracer.traced_grids_of_planes_from_grid(grid=self.masked_imaging.grid)[1]
  File "/home/carlos/anaconda3/lib/python3.7/site-packages/autoarray/structures/grids/decorators.py", line 157, in wrapper
    result_list = func(profile, grid, *args, **kwargs)
  File "/home/carlos/anaconda3/lib/python3.7/site-packages/autolens/lens/ray_tracing.py

KeyboardInterrupt: 

In [14]:
for i in range(10):
    log = model.log_probability(pars=pos[i])
    print("Iteração %i" %i)
    print("log_prob %f" %log)
    print("------------------------")

Iteração 0
log_prob -inf
------------------------
Iteração 1
log_prob -inf
------------------------


/home/carlos/anaconda3/lib/python3.7/site-packages/autogalaxy/profiles/mass_profiles/total_mass_profiles.py:98: RuntimeWarning: invalid value encountered in sqrt
  eta = np.sqrt(1 - q**2)
/home/carlos/anaconda3/lib/python3.7/site-packages/autogalaxy/profiles/mass_profiles/total_mass_profiles.py:120: RuntimeWarning: invalid value encountered in sqrt
  eta = np.sqrt(1 - q**2)


Iteração 2
log_prob -155261.550205
------------------------
Iteração 3
log_prob -285234.281284
------------------------
Iteração 4
log_prob -7471.878843
------------------------
Iteração 5
log_prob -inf
------------------------
Iteração 6
log_prob -6843.718856
------------------------
Iteração 7
log_prob -15308.657555
------------------------
Iteração 8
log_prob -inf
------------------------
Iteração 9
log_prob -inf
------------------------


In [12]:
i = 10
print("Iteração %i" %i)

Iteração 10


In [19]:
pos

array([ 9.90519845e+00,  9.38761809e+00,  4.56670065e+00,  3.95540285e+00,
        4.09138376e+00,  3.03500598e+00, -1.23701618e-01, -5.95326160e-01,
        6.70712457e-01, -2.71696684e+00,  1.28491888e+00, -9.11095039e-02,
        6.93551706e-01,  8.50485764e+01,  5.45598952e-01,  8.21609655e+00,
        9.16380573e+00,  2.62978391e-02,  3.44317093e-02, -8.18626727e-01])